# 首先，当然是“喜闻乐见”的环境配置环节
我的环境 win11 \
建议命令行操作

In [ ]:
!conda create -n mmpretrain python=3.9
# python版本和视频演示一致

# 安装最好带上-c nvidia 不然非常容易产生冲突，不知道咋形成的。
!conda activate mmpretrain
!conda install pytorch==2.0.1 torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia
# 不知道torchvision和torchaudia有不有用，先装了再说

# 克隆仓库，网页被重定向了就记得挂梯子
## 模块安装的好处：方便，简单，适合只是调用其中模块或者API
## 克隆好处：方便使用其中脚本，方便查看和修改其中模块，模型结构
!git clone https://github.com/open-mmlab/mmpretrain

# 安装MIM，这是OpenMMLab的包管理工具，用这个就犯不着冲突的问题。
!pip install openmim

# 安装mmpretrain算法库
!cd mmpretrain
!mim install -e ".[multimodal]"

# 查看安装是否成功
记一些报错：
1、显示没有安装ipykernel，因为我是用vscode，然后jupyter方式运行。 \
但是在弹出框点击“安装”和按照报错提示使用命令`conda install -n mmpretrain ipykernel --update-deps --force-reinstall`都没有效果，任然提示要求安装ipykernel。 \
解决方法：直接命令行输入`conda install ipykernel`就解决。\

2、报错“No module named 'importlib_metadata'”
直接`pip install importlib-metadata`即可。

In [2]:
# 查看安装的版本
import mmpretrain

mmpretrain.__version__

'1.0.0rc8'

get_model —— 模型的获取 \
list_models —— 模型的列举 \
inference_model —— 模型的推理 \

注意：get_model拿到的模型未加载预训练权重，不能直接用于模型推理

In [10]:
from mmpretrain import get_model, list_models, inference_model

# 列车图片分类任务中，resnet18模型有哪些
print(list_models(task='Image Classification', pattern='resnet18'))

print('-----------')

# 模型实例化
model = get_model('resnet18_8xb16_cifar10')
print(type(model))
print(type(model.backbone))

['resnet18_8xb16_cifar10', 'resnet18_8xb32_in1k']
-----------
<class 'mmpretrain.models.classifiers.image.ImageClassifier'>
<class 'mmpretrain.models.backbones.resnet_cifar.ResNet_CIFAR'>


In [14]:
# 模型需要传入预训练参数，以图片描述任务为例
print(list_models(task='Image Caption', pattern='blip'))
# 第二个参数是图片路径，我找了一个蜥蜴图片
inference_model('blip-base_3rdparty_caption', 'xiyi.jpg', show=True)


['blip-base_3rdparty_caption', 'blip2-opt2.7b_3rdparty-zeroshot_caption']


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'BlipTokenizer'.


Loads checkpoint by http backend from path: https://download.openmmlab.com/mmclassification/v1/blip/blip-base_3rdparty_coco-caption_20230419-a5b71af3.pth


{'pred_caption': 'a close up of a lizard on the ground'}

# 一个实例，猫和狗的分类任务
## 一、下载数据集
下载链接：（未给出）就不尝试了。

目录：
* training_set
* val_set
* test_set



In [ ]:
# 数据自己下载，然后解压到data文件夹下，data文件夹自己创建
# 查看一下目录结构 linux下

cd ./data/cats_dogs_dataset
tree ./ --filelimit=10

# 配置文件
configs目录下面是各种模型配置文件示例。\
配置文件的核心——用字典的形式进行所有配置的设置。 \
进入配置文件，发现关联了四个文件:
```
_base_ = [
    '../_base_/models/resnet18_cifar.py', '../_base_/datasets/cifar10_bs16.py',
    '../_base_/schedules/cifar10_bs128.py', '../_base_/default_runtime.py'
]
```
这里使用了OpenMMLab里面的继承机制，基层机制不详细讲解。\
可以了解的是，这里相当于**把四个配置未见里面的所有配置进行了综合**。

## model配置文件
可以查看一下`../_base_/models/resnet18_cifar.py`中的内容：

```python
# model settings
model = dict(

    # 用于定义model的类型
    type='ImageClassifier',

    # 主干网络，模型的核心
    backbone=dict(
        type='ResNet_CIFAR',
        depth=18,
        num_stages=4,
        out_indices=(3, ),
        style='pytorch'),
    
    # 颈，连接主干网络和头
    neck=dict(type='GlobalAveragePooling'),

    # 头，一般是模型最后的部分，例如有预分类头
    head=dict(
        type='LinearClsHead',

        # num_classes表示类别，如果二分类，就为2
        num_classes=10,
        in_channels=512,
        loss=dict(type='CrossEntropyLoss', loss_weight=1.0),
    ))

```


## 数据处理部分配置文件

# dataset settings
```python
dataset_type = 'CIFAR10'
data_preprocessor = dict(
    num_classes=10,
    # RGB format normalization parameters
    mean=[125.307, 122.961, 113.8575],
    std=[51.5865, 50.847, 51.255],
    # loaded images are already RGB format
    # 因为OpenCV得到的是BGR格式
    to_rgb=False)

train_pipeline = [
    dict(type='RandomCrop', crop_size=32, padding=4),
    dict(type='RandomFlip', prob=0.5, direction='horizontal'),
    dict(type='PackInputs'),
]

test_pipeline = [
    dict(type='PackInputs'),
]

# 数据集加载部分
train_dataloader = dict(
    batch_size=16,
    # 加载样本的进程数
    num_workers=2,
    # 训练数据集的配置
    dataset=dict(
        type=dataset_type,

        # 路径，注意一般训练集、测试集这些划分和数据集类型有关，也就是上面dataset_type = 'CIFAR10'字段
        data_root='data/cifar10',
        split='train',
        pipeline=train_pipeline),
    # 采样器的配置
    sampler=dict(type='DefaultSampler', shuffle=True),
)

val_dataloader = dict(
    batch_size=16,
    num_workers=2,
    dataset=dict(
        type=dataset_type,
        data_root='data/cifar10/',
        split='test',
        pipeline=test_pipeline),
    sampler=dict(type='DefaultSampler', shuffle=False),
)

# topk参数表示计算top-n的准确率，如果是二分类，就只能top-1准确率
val_evaluator = dict(type='Accuracy', topk=(1, ))

test_dataloader = val_dataloader
test_evaluator = val_evaluator
```

>>tips:
1. 如果已经加载过配置文件了，再修改`dataset_type`字段，那么下面`train_dataloader`里面的`dataset_type`并不会髓质改变，因为配置文件加载一次之后，字段之间的连接关系就没有了。



## 规划配置

schedules —— 训练、验证、测试的流程是怎么样的
```python

# 配置优化器，还可以在这里配置其他优化器参数
# optimizer
# 微调的话，可以把lr再拉小一点，比如0.01
optim_wrapper = dict(
    optimizer=dict(type='SGD', lr=0.1, momentum=0.9, weight_decay=0.0001))

# 指定在100，150个epoch处降低学习率
# learning policy
param_scheduler = dict(
    type='MultiStepLR', by_epoch=True, milestones=[100, 150], gamma=0.1)

# train, val, test setting
# 训练、验证、测试的流程设置
# 空表示按默认配置进行
# train_cfg里面的设置表示：整个训练流程总共进行200个epoch，每个epoch完成之后就进行一次验证
# 如果加载了预训练权重，那么训练几轮就好了，应为会收敛很快。
train_cfg = dict(by_epoch=True, max_epochs=200, val_interval=1)
val_cfg = dict()
test_cfg = dict()

# NOTE: `auto_scale_lr` is for automatically scaling LR
# based on the actual training batch size.
# batch_size越小，就需要越小的学习率。
# 注意如果是多卡，这里是所有PGU上所有batch_size加和
auto_scale_lr = dict(base_batch_size=128)

```

## 运行时间文件 

default_runtime.py
```python
# defaults to use registries in mmpretrain
default_scope = 'mmpretrain'

# configure default hooks
default_hooks = dict(
    # record the time of every iteration.
    timer=dict(type='IterTimerHook'),

    # print log every 100 iterations. interval代表间隔多少次迭代，打印一次日志。
    logger=dict(type='LoggerHook', interval=100),

    # enable the parameter scheduler.
    param_scheduler=dict(type='ParamSchedulerHook'),

    # save checkpoint per epoch. 没interval次，保留一次权重。
    # 还可以添加： max_kkep_ckpts=5, save_best='auto'  分别表示：保留最后5个checkpoints， 自动根据验证集上结果，保存到目前为止验证集上效果最好的模型
    checkpoint=dict(type='CheckpointHook', interval=1),

    # set sampler seed in distributed evrionment.
    sampler_seed=dict(type='DistSamplerSeedHook'),

    # validation results visualization, set True to enable it.
    visualization=dict(type='VisualizationHook', enable=False),
)

# configure environment
env_cfg = dict(
    # whether to enable cudnn benchmark
    cudnn_benchmark=False,

    # set multi process parameters
    mp_cfg=dict(mp_start_method='fork', opencv_num_threads=0),

    # set distributed parameters
    dist_cfg=dict(backend='nccl'),
)

# set visualizer
vis_backends = [dict(type='LocalVisBackend')]
visualizer = dict(type='UniversalVisualizer', vis_backends=vis_backends)

# set log level
log_level = 'INFO'

# load from which checkpoint
load_from = None

# whether to resume training from the loaded checkpoint
resume = False

# Defaults to use random seed and disable `deterministic`
# 一般在训练的时候就指定了随机种子， deterministic是确定性增强选项，
randomness = dict(seed=None, deterministic=False)

```

# 实战

## 第一步，配置文件
可以直接将上述配置文件中内容复制粘贴到我们的配置文件，这样所需要的配置参数就得到了。  
然后，根据需求修改配置文件：
1. 加载预训练权重。  —— 其实就是一种特殊的模型参数初始化的方式。  
>> tips：预训练权重：模型在其他大型数据集上，进行有监督或者无监督的方式提前训练好的参数。虽然，预训练使用的数据集和我们目标数据集不一样，但是模型学习到的提取图像特征是共通的。这样可以大大加速模型在我们目标训练集上的收敛速度。  

模型预训练权重哪里可以找到：在MMPretrain的[官网](https://mmpretrain.readthedocs.io/en/latest/)，“model zoo”一栏，按照需求右键复制所需模型的下载链接。

在model字典里面修改添加字段：
```python
init_cfg=dict(type='Pretrained', checkpoint='https://download.openmmlab.com/mmclassification/v0/beit/beit-base_3rdparty_in1k_20221114-c0a4df23.pth')
```

2. 根据需求修改配置文件

##  开启训练
1. 方式一：使用tools/下面的train文件。
2. 可以使用`mim`在目录中任意位置启动训练：\
`mim train mmpretrain 配置文件.py --work-dir=./exp` --work-dir参数指定保存路径。  

## 开启测试
`mim test mmpretrain 配置文件.py --checkpoint 模型路径`

保留每个测试样本的结构到result.pkl文件里\
`mim test mmpretrain 配置文件.py --checkpoint 模型路径 --out result.pkl`

## 自带工具分析结果
分析哪些是得分很高而且是正确的，哪些得分很高但是分类错误\
`mim run mmpretrain analyze_results 配置文件.py result.pkl --out-dir analyze`

得到混淆矩阵\
`mim run mmpretrain confusion_matrix 配置文件.py result.pkl --show --include-values`

# 最后，在一个真实图片上进行推理

In [ ]:
from mmpretrain import ImageClassificationInferencer

inferencer = ImageClassificationInferencer('配置文件.py', pretrained='模型路径')
inferencer("图片路径", show=True)